# Web Scraping with lazy load

Page to be scraped: El Espectador

In [15]:
pip install webdriver

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement webdriver (from versions: none)
ERROR: No matching distribution found for webdriver
You should consider upgrading via the 'C:\Users\Administrador\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
# Importing required libraries

import time

from bs4 import BeautifulSoup
from selenium import webdriver

from pymongo import MongoClient


In [2]:

# Creating a connection to MongoDB
client = MongoClient("localhost", 27017)
db = client['news']
collection = db['elespectador']


In [3]:
# Base URL of the site to be analyzed
SITE_URL = 'https://www.elespectador.com'

In [4]:
# Firefox web driver path
# Download the driver for you S.O. here: https://github.com/mozilla/geckodriver/releases
DRIVER_PATH = './geckodriver.exe'

In [5]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

C:\Users\Administrador\AppData\Local\Temp\ipykernel_16068\1910320475.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [6]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [96]:
# Getting HTML content for a particular news listing page
soup = make_request(browser, '/archivo/politica/')

In [7]:
def urls(browser, data):
    soup = make_request(browser, data)
    layout = soup.find(class_ = 'Layout-flexAds')
    blocks = layout.find_all(class_ = 'Container Block', recursive = True)

    print(len(blocks))
    print(blocks)
    if(len(blocks)  == 2):
        cards = blocks[0].find_all(class_ = 'Card_rowCardLeft') + blocks[1].find_all(class_ = 'Card_rowCardLeft')
        len(cards)
    elif(len(blocks)  == 1):
        cards = blocks[0].find_all(class_ = 'Card_rowCardLeft')
        len(cards)
    
    news = []

    for card in cards:
         news.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
         })
    for n in news:
        # Getting HTML content for each news page
        soup = make_request(browser, n['relative_path'])
    
        # Extracting news metadata
        n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
        n['author'] = soup.find(class_ = 'ACredit-Author').find('a').get_text()
        n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
        # Extracting and concatenating news full text
        paragraphs = soup.find_all(class_ = 'font--secondary')
        n['full_text'] = ' '.join([p.get_text() for p in paragraphs])
    news
    collection.insert_many(news)

    
    

In [8]:
print("entro")
lista = ['/archivo/judicial/20/','/archivo/bogota/32/','/archivo/mundo/32/','/archivo/educacion/32/']
for v in lista:
    print(v)
    urls(browser,v)



entro
/archivo/judicial/20/
2
[<div class="Container Block"><div class="Grid pure-g"><div class="false pure-u-1"></div><div class="Grid pure-g pure-u-1"><div class="false BlockMargin_bottomBorders"><div class="Card Card_rowCardLeft padding-Border"><div class="Card-ImageFrame"><div class="Card-ImageLink"><a href="/judicial/a-mitad-de-la-jornada-electoral-van-20-capturados-y-32-denuncias-de-compra-de-voto/" rel="noreferrer" target="_self"><picture><source class="Card-Image" media="(max-width: 1200px)" srcset="https://www.elespectador.com/resizer/_Z11jVl0ZfYXzeM7wmsj7egWxuI=/180x120/filters:focal(3607x1205:3787x1325):format(jpeg)/cloudfront-us-east-1.images.arcpublishing.com/elespectador/SMFTG5FKUXAB6IM2IYEKPM7QTE.jpg"/><img alt="A mitad de la jornada electoral van 20 capturados y 32 denuncias de compra de voto" class="Card-Image" src="https://www.elespectador.com/resizer/_Z11jVl0ZfYXzeM7wmsj7egWxuI=/180x120/filters:focal(3607x1205:3787x1325):format(jpeg)/cloudfront-us-east-1.images.arcpu

AttributeError: 'NoneType' object has no attribute 'get_text'

In [84]:
# Finding the section where news are contained 
layout = soup.find(class_ = 'Layout-flexAds')

In [85]:
# Getting blocks from layout
blocks = layout.find_all(class_ = 'Container Block', recursive = True)

print(len(blocks))
print(blocks)

1
[<div class="Container Block"><div class="Grid pure-g"><div class="false pure-u-1"></div><div class="Grid pure-g pure-u-1"><div class="false BlockMargin_bottomBorders"><div class="Card Card_rowCardLeft padding-Border"><div class="Card-ImageFrame"><div class="Card-ImageLink"><a href="/politica/critican-decision-de-duque-de-impedir-intervencion-civil-en-consejo-de-seguridad/" rel="noreferrer" target="_self"><picture><source class="Card-Image" media="(max-width: 1200px)" srcset="https://www.elespectador.com/resizer/kUI_WqdJljTWfMbYZTJWzNR5AIM=/287x191/filters:format(jpeg)/cloudfront-us-east-1.images.arcpublishing.com/elespectador/FH3B2R7W3CVD62E6AXAEMK34YY.jpg"/><img alt="Critican decisión de Duque de impedir intervención civil en Consejo de Seguridad" class="Card-Image" src="https://www.elespectador.com/resizer/kUI_WqdJljTWfMbYZTJWzNR5AIM=/287x191/filters:format(jpeg)/cloudfront-us-east-1.images.arcpublishing.com/elespectador/FH3B2R7W3CVD62E6AXAEMK34YY.jpg"/></picture></a></div></div><

In [90]:
# Finding and concatenating news cards

if(len(blocks)  == 2):
    cards = blocks[0].find_all(class_ = 'Card_rowCardLeft') + blocks[1].find_all(class_ = 'Card_rowCardLeft')
    len(cards)
elif(len(blocks)  == 1):
    cards = blocks[0].find_all(class_ = 'Card_rowCardLeft')
    len(cards)



In [91]:
# Building a list with title and relative path of the news founded

news = []

for card in cards:
    news.append({
        'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
        'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
    })

In [92]:
news

[{'title': 'Critican decisión de Duque de impedir intervención civil en Consejo de Seguridad',
  'relative_path': '/politica/critican-decision-de-duque-de-impedir-intervencion-civil-en-consejo-de-seguridad/'},
 {'title': 'Fajardo pide garantías de imparcialidad de la Fiscalía por lo del Putumayo ',
  'relative_path': '/politica/elecciones-colombia-2022/fajardo-pide-garantias-de-imparcialidad-de-la-fiscalia-por-lo-del-putumayo/'},
 {'title': 'Análisis: los partidos no están fomentando la inclusión de las mujeres',
  'relative_path': '/politica/elecciones-colombia-2022/los-partidos-no-estan-fomentando-la-inclusion-de-las-mujeres/'},
 {'title': '“Son irresponsables”: Flip sobre mensajes de Enrique Gómez contra periodistas',
  'relative_path': '/politica/elecciones-colombia-2022/son-altamente-irresponsables-flip-sobre-mensajes-de-enrique-gomez-contra-periodistas/'}]

In [93]:
for n in news:
    # Getting HTML content for each news page
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').find('a').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [94]:
news

[{'title': 'Critican decisión de Duque de impedir intervención civil en Consejo de Seguridad',
  'relative_path': '/politica/critican-decision-de-duque-de-impedir-intervencion-civil-en-consejo-de-seguridad/',
  'datetime': '12 abr. 2022  - 7:02 a.\xa0m.',
  'author': 'Redacción Política',
  'summary': 'Varias organizaciones llamaron la atención frente al pedido que les impediría participar este martes en la rendición de cuentas trimestral de los avances del proceso de paz.',
  'full_text': 'A través de una carta, varias organizaciones de víctimas y víctimas a nombre propio llamaron la atención del Consejo de Seguridad frente a la supuesta petición del presidente Iván Duque para que la sociedad civil no pueda participar de la rendición de cuentas trimestral del acuerdo de paz colombiano, durante la sesión de esta célula de las Naciones Unidas. Son más de 92 organizaciones las firmantes que advirtieron que la medida pedida por el primer mandatario colombiano pone en peligro la implementa

In [95]:
# Storing extracted information for further analysis
collection.insert_many(news)